In [1]:
# Install pyspark and findspark
!pip install --ignore-install -q pyspark
# Install findspark library
!pip install --ignore-install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 16.2 MB/s eta 0:00:00


In [2]:
# Import findspark
import findspark
findspark.init()

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("logregdoc").getOrCreate()
# Load training data
data_set = spark.read.format("libsvm").option("numFeatures","692").load('/content/drive/MyDrive/sample_libsvm_data.txt')
data_set.show()
# Split into training set and testing set
lr_train, lr_test = data_set.randomSplit([0.7,0.3])
# Create LogisticRegression model
lgr_model = LogisticRegression()
# fit model with training set
fitted_model = lgr_model.fit(lr_train)
# evaluate with testing set
prediction_and_labels = fitted_model.evaluate(lr_test)
prediction_and_labels.predictions.show()
prediction_and_labels.predictions.select("probability").show(10, False)

my_eval = BinaryClassificationEvaluator()
final_roc = my_eval.evaluate(prediction_and_labels.predictions)
print(final_roc)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[35.

In [5]:
spark.stop()